In [ ]:

%load_ext autoreload 
%autoreload 2
%matplotlib widget
%matplotlib inline
import cProfile #for checking the nr of calls and execution time
import pstats
from pstats import SortKey

from tqdm.notebook import tqdm
import numpy as np
import multiple_planets_gas_acc as code_gas
from functions_pebble_accretion import *
from functions import *
import functions_plotting as plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
import pandas as pd
from matplotlib.ticker import ScalarFormatter, LogFormatter, LogLocator, MultipleLocator, AutoMinorLocator
from matplotlib import cm, ticker
from matplotlib import colors
import matplotlib.gridspec as gridspec
import matplotlib.patches as patch
from matplotlib.offsetbox import AnchoredText
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.lines as mlines 

color = mpl.colormaps["YlOrRd"].reversed()(np.linspace(0, 0.7, code_gas.sim_params.nr_planets))


In [ ]:

# disc parameters
params_dict = {'St_const': None, 
                'M_dot_star': None,
                'v_frag':(1 * u.m/u.s).to(u.au/u.Myr).value,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'iceline_alpha_change': False,
                'iceline_flux_change': False,
                }

t_initial = 0.2
a_p0 = np.geomspace(2,2, num = 1)
t0= [t_initial] * np.ones(len(a_p0))# warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
t_in = 0.2
t_fin = 10
sim_params_dict = {'N_step': 10000,
                'a_p0':a_p0,
                't0':t0,
                't_in':t_in,
                't_fin':t_fin
                }

params_ida_ext_2 = code_gas.Params(**params_dict, H_r_model='irradiated')
# params_ida_ext_3 = code_gas.Params(**params_dict, H_r_model='Ida_mixed', epsilon_el=1, epsilon_heat=1, alpha =1e-3)

# params_lam_ext_2 = code_gas.Params(**params_dict, H_r_model='Lambrechts_mixed', epsilon_el=1e-2, epsilon_heat=0.5, alpha =1e-2)
# params_lam_ext_3 = code_gas.Params(**params_dict, H_r_model='Lambrechts_mixed', epsilon_el=1e-2, epsilon_heat=0.5, alpha =1e-3)


# m0_lam_ext_2 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_lam_ext_2), params_lam_ext_2)
# sim_params_lam_ext_2 = code_gas.SimulationParams(**sim_params_dict, m0 = m0_lam_ext_2)
# m0_lam_ext_3 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_lam_ext_3), params_lam_ext_3)
# sim_params_lam_ext_3 = code_gas.SimulationParams(**sim_params_dict, m0 = m0_lam_ext_3)

m0_ida_ext_2 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_ida_ext_2), params_ida_ext_2)
sim_params_ida_ext_2 = code_gas.SimulationParams(**sim_params_dict, m0 = m0_ida_ext_2)
# m0_ida_ext_3 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_ida_ext_3), params_ida_ext_3)
# sim_params_ida_ext_3 = code_gas.SimulationParams(**sim_params_dict, m0 = m0_ida_ext_3)

peb_acc = code_gas.PebbleAccretion(simplified_acc=False)
gas_acc = code_gas.GasAccretion()

# sim_lam_ext_2 = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_lam_ext_2, sim_params=sim_params_lam_ext_2)
# sim_lam_ext_3 = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_lam_ext_3, sim_params=sim_params_lam_ext_3)
# sim_ida_ext_2 = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_ida_ext_2, sim_params=sim_params_ida_ext_2)
# sim_ida_ext_3 = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_ida_ext_3, sim_params=sim_params_ida_ext_3)

sim_ida_ext_2 = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_ida_ext_2, sim_params=sim_params_ida_ext_2)

# ## to run the sim and profile execution time and nr of calls
# def run_simulation():
#     sim_ida_ext_2 = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_ida_ext_2, sim_params=sim_params_ida_ext_2)
#     #sim_ida_ext_3 = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_ida_ext_3, sim_params=sim_params_ida_ext_3)

# # Profile the function
# profiler = cProfile.Profile()
# profiler.enable()
# run_simulation()
# profiler.disable()

# # Create a Stats object and sort the output by total time
# stats = pstats.Stats(profiler).sort_stats(SortKey.TIME)

# # Print the top 10 functions that take the longest time
# stats.print_stats(10)

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(6, 6))
axs.loglog(sim_ida_ext_2.time, sim_ida_ext_2.sigma_gas[0])
print(sim_ida_ext_2.sigma_gas[0])

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(5, 5))
pos = np.geomspace(1e-2, 100, num = 10)
t = np.geomspace(1e-1, 10, num = 10)
t0 = 0.1
params = code_gas.Params(**params_dict)
axs.loglog(pos, H_R_irr(pos, params_ida_ext_2))

In [ ]:
erg_s_to_au_M_E_Myr = (1*u.erg/u.s).to(u.au**2*u.M_earth/u.Myr**3).value
L = (const.L_sun.cgs).to(u.au**2*u.M_earth/u.Myr**3)
M = (const.M_sun).to(u.M_earth)
r = 1*u.au
print((const.L_sun.cgs).to(u.au**2*u.M_earth/u.Myr**3))
print((const.M_sun).to(u.M_earth))
params = code_gas.Params(**params_dict, star_luminosity= L, star_mass = M)
print( "in au",0.024*(L)**(1/7)*(M)**(-4/7)*(r)**(2/7))
print(H_R_irr(1, params))

In [ ]:
fig,axs = plt.subplots(2,2, figsize = (18,18))
label = ["no filtering", "filtering"]


# plot.plot_growth_track_timescale(fig, axs[0], sim_filter_irr, params_irr, sim_params_irr, migration = True,
#                                     cmap = mpl.colormaps["inferno"].reversed(), add_ylabel=True , 
#                                         add_cbar= True , nofilter=True if i ==0 else False
#                                         )

# plot.plot_growth_track_timescale(fig, axs[0,0], sim_lam_ext_3, params_lam_ext_3, sim_params_lam_ext_3, migration = True,
#                                 cmap = mpl.colormaps["inferno"].reversed(), add_ylabel= False, 
#                                     add_cbar=False, nofilter=False
#                                     )

# plot.plot_growth_track_timescale(fig, axs[0,1], sim_lam_ext_2, params_lam_ext_2, sim_params_lam_ext_2, migration = True,
#                                 cmap = mpl.colormaps["inferno"].reversed(), add_ylabel= False, 
#                                     add_cbar=True, nofilter=False
#                                     )

# plot.plot_growth_track_timescale(fig, axs[1,0], sim_ida_ext_3, params_ida_ext_3, sim_params_ida_ext_3, migration = True,
#                                 cmap = mpl.colormaps["inferno"].reversed(), add_ylabel= False, 
#                                     add_cbar=False, nofilter=False
#                                     )

plot.plot_growth_track_timescale(fig, axs[1,1], sim_ida_ext_2, params_ida_ext_2, sim_params_ida_ext_2, migration = True,
                                cmap = mpl.colormaps["inferno"].reversed(), add_ylabel= False, 
                                    add_cbar=True, nofilter=False
                                    )
for p in range(sim_params_ida_ext_2.nr_planets-1,-1,-1):
    isolation_mass = M_peb_iso(sim_ida_ext_2.position[p].value, sim_ida_ext_2.time.value, params_ida_ext_2)
    iso_idx = np.argmax(sim_ida_ext_2.mass[p].value > isolation_mass)
    axs[1,0].axvline(sim_ida_ext_2.time[iso_idx].value, color = 'lightblue', linestyle = '--')

axs[1,0].scatter(sim_ida_ext_2.time.to(u.Myr), sim_ida_ext_2.dR_dt[0], color = 'black')
axs[1,0].set_xscale('log')
axs[1,0].set_xlabel('t [Myr]', size = 20)
axs[1,0].set_ylabel('$\dot{R}$ [AU/Myr]', size = 20)

axs[0,1].scatter(sim_ida_ext_2.position.to(u.au)[0], sim_ida_ext_2.dR_dt[0], color = 'black')
axs[0,1].set_xscale('log')
axs[0,1].set_xlabel('R [AU]', size = 20)
axs[0,1].set_ylabel('$\dot{R}$ [AU/Myr]', size = 20)
print("dr/dt", sim_ida_ext_2.dR_dt[0])
print("r/rdot", sim_ida_ext_2.position.to(u.au)[0]/sim_ida_ext_2.dR_dt[0])
print("step size [yr]", (sim_params_ida_ext_2.step_size*u.Myr))
print("time vector[Myr]", (sim_ida_ext_2.time*u.Myr))
print(sim_ida_ext_2.time.size)
print(np.where(sim_params_ida_ext_2.step_size*u.Myr>(sim_ida_ext_2.position[0]/np.abs(sim_ida_ext_2.dR_dt[0])))[0])
print("min incremento", np.min(sim_ida_ext_2.position[0]/np.abs(sim_ida_ext_2.dR_dt[0])))
print("min incremento massa", np.min(sim_ida_ext_2.mass[0]/np.abs(sim_ida_ext_2.dM_dt[0])))

axs[0,0].scatter(sim_ida_ext_2.time.to(u.Myr), sim_ida_ext_2.dM_dt[0], color = 'black')
axs[0,0].set_xscale('log')
axs[0,0].set_xlabel('t [Myr]]', size = 20)
axs[0,0].set_ylabel('$\dot{M}$ [$M_{\oplus}$/Myr]', size = 20)
axs[0,0].axhline(sim_params_ida_ext_2.step_size, color = 'red', linestyle = '--')
axs[0,0].set_xlim(0.2, 0.3)
axs[1,0].set_xlim(0.2, 0.3)
for i in range(2):
    for j in range(2):
        axs[i,j].tick_params(axis = "both", which = "major", direction = 'in', size = 15)
        axs[i,j].tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
        #axs[i,j].set_xlim(4e-3, 1e2)
        # plot.kepler11(axs[i,j], color = 'black')
        # plot.solar_system(axs[i,j], color = 'green')



plt.savefig("figures/t_fixed_adaptive_2au", bbox_inches='tight')

## DIFFERENT MODELS PLOT

In [ ]:
# disc parameters
params_dict = {'St_const': None, 
                'M_dot_star': None,
                'iceline_radius': None,
                'v_frag':(1*u.m/u.s).to(u.au/u.Myr).value,
                'alpha': 1e-2,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                }

t_in = 0.2
t_fin = 5
# a_p0_in = np.geomspace(3, 0.1, num=5)
# # Combine the single value 20 with the generated array
# a_p0 = np.concatenate(([20], a_p0_in))
a_p0 = np.geomspace(60, 0.1, num = 5)
print(a_p0)
t0= ([t_in] * np.ones(len(a_p0))) # warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work

sim_params_dict = {'N_step': 10000,
                'a_p0':a_p0,
                't0':t0,
                't_in':t_in,
                't_fin': t_fin
                }

# params_flux = code_gas.Params(**params_dict, H_r_model='irradiated', iceline_flux_change=True, iceline_alpha_change=False, iceline_alpha_frag_change=False)
# params_alpha_flux = code_gas.Params(**params_dict, H_r_model='irradiated', iceline_flux_change=True, iceline_alpha_change=True, iceline_alpha_frag_change=False)
# params_alpha_flux_alphafrag = code_gas.Params(**params_dict, H_r_model='irradiated', iceline_flux_change=True, iceline_alpha_change=True, iceline_alpha_frag_change=True)

params_flux = code_gas.Params(**params_dict, H_r_model='irradiated')
params_alpha_flux = code_gas.Params(**params_dict, H_r_model='Lambrechts_mixed', epsilon_el=1e-2, epsilon_heat=0.5)
params_alpha_flux_alphafrag = code_gas.Params(**params_dict, H_r_model='Lambrechts_mixed', epsilon_el=1, epsilon_heat=1)

m0 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_flux), params_flux)
sim_params = code_gas.SimulationParams(**sim_params_dict, m0 = m0)

m02 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_alpha_flux), params_alpha_flux)
sim_params2 = code_gas.SimulationParams(**sim_params_dict, m0 = m02)

m03 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_alpha_flux_alphafrag), params_alpha_flux_alphafrag)
sim_params3 = code_gas.SimulationParams(**sim_params_dict, m0 = m03)


peb_acc = code_gas.PebbleAccretion(simplified_acc=False)
gas_acc = code_gas.GasAccretion()

sim_filter_flux = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_flux, sim_params=sim_params)
sim_filter_alpha_flux = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_alpha_flux, sim_params=sim_params2)
sim_filter_alpha_flux_alphafrag = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_alpha_flux_alphafrag, sim_params=sim_params3)


In [ ]:
sim_filter_flux_nofil = code_gas.simulate_euler(migration=True, filtering=False, peb_acc=peb_acc, gas_acc=gas_acc, params=params_flux, sim_params=sim_params)
sim_filter_alpha_flux_nofil = code_gas.simulate_euler(migration=True, filtering=False, peb_acc=peb_acc, gas_acc=gas_acc, params=params_alpha_flux, sim_params=sim_params2)
sim_filter_alpha_flux_alphafrag_nofil = code_gas.simulate_euler(migration=True, filtering=False, peb_acc=peb_acc, gas_acc=gas_acc, params=params_alpha_flux_alphafrag, sim_params=sim_params3)


In [ ]:
fig, axs = plt.subplots(1,3, figsize = (18,6))

ls = ['--','-']
#title = ["$F_{\mathrm{ice}} = 50 \% F_0 $", "$F_{\mathrm{ice}} = 50 \% F_0 $ + "+r"$\alpha_z = 10^{-3}$ ", "$F_{\mathrm{ice}} = 50 \% F_0 $ + "+r"$\alpha_z, \alpha_{\mathrm{frag}} = 10^{-3}$"]
title = ["Irradiated disc", "Surface-heated disc", "Midplane-heated disc"]

cmap = mpl.colormaps["YlOrRd"].reversed()(np.linspace(0, 0.7, code_gas.sim_params.nr_planets))
# # nofilter for 3 plots
# for i, (sim, params) in enumerate(zip([sim_filter_flux_nofil, sim_filter_alpha_flux_nofil, sim_filter_alpha_flux_alphafrag_nofil],[params_flux, params_alpha_flux, params_alpha_flux_alphafrag])): #sim_filter_alpha_flux, sim_filter_alpha_flux_alphafrag], [params_flux, params_alpha_flux, params_alpha_flux_alphafrag])):
#     plot.plot_growth_track_timescale(fig, axs[i], sim, params, sim_params,  migration=True, cmap=mpl.colormaps["inferno"].reversed(), add_ylabel=True if i ==0 else False,
#                                       add_cbar=False, nofilter=True)

# nofilter for just the third plot
for i, (sim, params) in enumerate(zip([ sim_filter_alpha_flux_alphafrag_nofil],[params_alpha_flux_alphafrag])): #sim_filter_alpha_flux, sim_filter_alpha_flux_alphafrag], [params_flux, params_alpha_flux, params_alpha_flux_alphafrag])):
    plot.plot_growth_track_timescale(fig, axs[2], sim, params, sim_params,  migration=True, cmap=mpl.colormaps["inferno"].reversed(), add_ylabel=False,
                                      add_cbar=True, nofilter=True)

# filter for the 3 plots
for i, (sim, params) in enumerate(zip([sim_filter_flux, sim_filter_alpha_flux, sim_filter_alpha_flux_alphafrag],[params_flux, params_alpha_flux, params_alpha_flux_alphafrag])): #sim_filter_alpha_flux, sim_filter_alpha_flux_alphafrag], [params_flux, params_alpha_flux, params_alpha_flux_alphafrag])):
    plot.plot_growth_track_timescale(fig, axs[i], sim, params, sim_params,  migration=True, cmap=mpl.colormaps["inferno"].reversed(), add_ylabel=True if i ==0 else False,
                                      add_cbar=True if i ==0 else False, nofilter=False)
    axs[i].set_title(title[i], fontsize = 25)
    pos = np.geomspace(4e-3, 1e2, num = 1000)
    M_iso_fin=M_peb_iso(pos, sim_params.t_fin, params)
    M_iso_in=M_peb_iso(pos, sim_params.t_in, params)
    axs[i].fill_between(pos, M_iso_fin, M_iso_in,  color='slateblue', alpha=0.1, zorder=10)
    # axs[i].axvline(iceline(sim_params.t_fin, 170, params), color= 'lightblue', linestyle = '-.')
    # axs[i].axvline(iceline(sim_params.t_in, 170, params), color= 'lightblue', linestyle = '-.')
    # axs[i].axvspan(iceline(sim_params.t_in, 170, params), iceline(sim_params.t_fin, 170, params),  color='lightblue', alpha=0.1)

for i in range(3):
    axs[i].tick_params(axis = "both", which = "major", direction = 'in', size = 15)
    axs[i].tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
    plot.HD219134(axs[i], color = 'red')
    plot.solar_system(axs[i], color = 'green')

fig.subplots_adjust(wspace=0.25)  # Increase the value to add more space

plt.savefig("figures/5_planets_filter_nofilter_5Myr", bbox_inches = 'tight')

## ICELINE PLOT 

In [ ]:
# disc parameters
params_dict = {'St_const': None, 
                'M_dot_star': None,
                'iceline_radius': None,
                'v_frag':(1*u.m/u.s).to(u.au/u.Myr).value,
                'alpha': 1e-2,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                }

t_in = 0.2
t_fin = 5
# a_p0_in = np.geomspace(3, 0.1, num=5)
# # Combine the single value 20 with the generated array
# a_p0 = np.concatenate(([20], a_p0_in))
a_p0 = np.geomspace(60, 0.1, num = 5)
print(a_p0)
t0= ([t_in] * np.ones(len(a_p0))) # warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work

sim_params_dict = {'N_step': 10000,
                'a_p0':a_p0,
                't0':t0,
                't_in':t_in,
                't_fin': t_fin
                }

params_flux = code_gas.Params(**params_dict, H_r_model='irradiated', iceline_flux_change=True, iceline_alpha_change=False, iceline_alpha_frag_change=False)
params_alpha_flux = code_gas.Params(**params_dict, H_r_model='irradiated', iceline_flux_change=True, iceline_alpha_change=True, iceline_alpha_frag_change=False)
params_alpha_flux_alphafrag = code_gas.Params(**params_dict, H_r_model='irradiated', iceline_flux_change=True, iceline_alpha_change=True, iceline_alpha_frag_change=True)

# params_flux = code_gas.Params(**params_dict, H_r_model='irradiated')
# params_alpha_flux = code_gas.Params(**params_dict, H_r_model='Lambrechts_mixed', epsilon_el=1e-2, epsilon_heat=0.5)
# params_alpha_flux_alphafrag = code_gas.Params(**params_dict, H_r_model='Lambrechts_mixed', epsilon_el=1, epsilon_heat=1)

m0 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_flux), params_flux)
sim_params = code_gas.SimulationParams(**sim_params_dict, m0 = m0)

m02 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_alpha_flux), params_alpha_flux)
sim_params2 = code_gas.SimulationParams(**sim_params_dict, m0 = m02)

m03 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_alpha_flux_alphafrag), params_alpha_flux_alphafrag)
sim_params3 = code_gas.SimulationParams(**sim_params_dict, m0 = m03)


peb_acc = code_gas.PebbleAccretion(simplified_acc=False)
gas_acc = code_gas.GasAccretion()

sim_filter_flux = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_flux, sim_params=sim_params)
sim_filter_alpha_flux = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_alpha_flux, sim_params=sim_params2)
sim_filter_alpha_flux_alphafrag = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_alpha_flux_alphafrag, sim_params=sim_params3)


In [ ]:
fig, axs = plt.subplots(1,3, figsize = (18,6))

ls = ['--','-']
title = ["$F_{\mathrm{ice}} = 50 \% F_0 $", "$F_{\mathrm{ice}} = 50 \% F_0 $; "+r"$\alpha_z = 10^{-3}$ ", "$F_{\mathrm{ice}} = 50 \% F_0 $; "+r"$\alpha_z, \alpha_{\mathrm{frag}} = 10^{-3}$"]

cmap = mpl.colormaps["YlOrRd"].reversed()(np.linspace(0, 0.7, code_gas.sim_params.nr_planets))

# filter for the 3 plots
for i, (sim, params) in enumerate(zip([sim_filter_flux, sim_filter_alpha_flux, sim_filter_alpha_flux_alphafrag],[params_flux, params_alpha_flux, params_alpha_flux_alphafrag])): #sim_filter_alpha_flux, sim_filter_alpha_flux_alphafrag], [params_flux, params_alpha_flux, params_alpha_flux_alphafrag])):
    plot.plot_growth_track_timescale(fig, axs[i], sim, params, sim_params,  migration=True, cmap=mpl.colormaps["inferno"].reversed(), add_ylabel=True if i ==0 else False,
                                      add_cbar=True if i ==0 else False, nofilter=False)
    axs[i].set_title(title[i], fontsize = 25)
    pos = np.geomspace(4e-3, 1e2, num = 1000)
    M_iso_fin=M_peb_iso(pos, sim_params.t_fin, params)
    M_iso_in=M_peb_iso(pos, sim_params.t_in, params)
    axs[i].fill_between(pos, M_iso_fin, M_iso_in,  color='slateblue', alpha=0.1, zorder=10)
    axs[i].axvline(iceline(sim_params.t_fin, 170, params), color= 'lightblue', linestyle = '-.')
    axs[i].axvline(iceline(sim_params.t_in, 170, params), color= 'lightblue', linestyle = '-.')
    axs[i].axvspan(iceline(sim_params.t_in, 170, params), iceline(sim_params.t_fin, 170, params),  color='lightblue', alpha=0.1)

for i in range(3):
    axs[i].tick_params(axis = "both", which = "major", direction = 'in', size = 15)
    axs[i].tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
    plot.HD219134(axs[i], color = 'red')
    plot.solar_system(axs[i], color = 'green')

fig.subplots_adjust(wspace=0.25)  # Increase the value to add more space

plt.savefig("figures/5_planets_iceline_5Myr", bbox_inches = 'tight')